# COGS 108 - Data Checkpoint

## Ava Emani, Joshua Choi, Hattan Halawani, Neha Ayyappan, Kelly Sam

- Ava Emani: Project administration, Software, Review and editing
- Joshua Choi: Software, Analysis, Data Curation
- Hattan Halawani: Research development, Data analysis
- Neha Ayyappan: Hypothesis development, Ethics section, Background review
- Kelly Sam: Background review, Writing and review

## Research Question

How do borrower income, loan-to-income ratio, employment length, home ownership status, loan intent, prior default history, and credit history affect the probability of a loan being defaulted?

The study focuses on the correlation between the binary outcome variable loan_status, where loan_status =1 indicates a borrower has failed to pay off (defaulted), while the loan_status= 0 indicates repayment as agreed. Utilizing numerous descriptive statistics, such as with mean and median, the study attempts to summarize borrower and loan characteristics from the data set as well as use group comparisons via calculating the default rate across the categories of home ownership and loan intent to better be able to identify difference in default risk between groups while applying logistic regression, to model to the audience the probability of loan default as a function to person_income, loan_percent_income, person_emp_length,person_home_ownership, cb_person_default_on_file, cb_person_cred_hist_length and loan_intent and whether the variables play a factor in the probability of the binary outcome variable loan_status being defaulted.

## Background and Prior Work

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback

## Hypothesis


Borrowers with low annual income, higher loan-to-income percentages, shorter employment length, non-homeownership status, certain loan intents, prior default history, and shorter credit history are more likely to default on loans.

These factors affect the borrower's overall financial stability and credit risk profile. Lower income and higher loan-to-income percentages indicate financial strain while shorter employment length, non-homeownership status, prior default history, and shorter credit history suggest increased financial vulnerability. When combined, these factors increase the likelihood that borrowers will miss loan payments and defaults.

## Data

### Data overview

- Dataset #1
  - Dataset Name: Credit Risk Dataset
  - Link to the dataset: https://www.kaggle.com/datasets/laotse/credit-risk-dataset
  - Number of observations: 32,581
  - Number of variables: 12
  - Description of the variables most relevant to this project
    - ```person_age```: A person's age in years
    - ```person_income```: A person's annual income is USD 
    - ```loan_int_rate```: A person's interest rate (%)
    - ```loan_status```: Our target variable. ```0``` indicates non-default and ```1``` indicates default
  - Descriptions of any shortcomings this dataset has with repsect to the project
    - This dataset has some near impossible outliers like age being too high physically, too long of years of work experience, etc. These outliers must be removed to prevent skewness for our models. There are some missing data entries for some variables and we will have to clean that up. We decided to use the median imputation for loan interest rates since the distribution may be skewed. The median can provide a typical value and not be influenced by extreem outliers. 

In [ ]:
# Run this code every time when you're actively developing modules in .py files.  It's not needed if you aren't making modules
#
## this code is necessary for making sure that any modules we load are updated here 
## when their source code .py files are modified

%load_ext autoreload
%autoreload 2

In [ ]:
# Setup code -- this only needs to be run once after cloning the repo!
# this code downloads the data from its source to the `data/00-raw/` directory
# if the data hasn't updated you don't need to do this again!

# if you don't already have these packages (you should!) uncomment this line
# %pip install requests tqdm

import sys
sys.path.append('./modules') # this tells python where to look for modules to import

import get_data # this is where we get the function we need to download data

# replace the urls and filenames in this list with your actual datafiles
# yes you can use Google drive share links or whatever
# format is a list of dictionaries; 
# each dict has keys of 
#   'url' where the resource is located
#   'filename' for the local filename where it will be stored 
datafiles = [
    { 'url': 'https://raw.githubusercontent.com/LeHongNgoc3820/Project_Credit_Risk/main/credit_risk_dataset.csv', 'filename':'credit_risk_dataset.csv'},
]

get_data.get_raw(datafiles,destination_directory='data/00-raw/')

### Credit Risk Dataset

The dataset available throgh Kaggle is utilized for its financial and demographic information to predict whether an indivdual would have their loan defaulted (i.e., not paid off). Each row represents a loan applicant and the primary outcome variable (loan_status) indicated whther the borrower has repaid the loan successfully (coded as 0) or defaulted (coded as 1). With this dataset the group sets to structurally utilize supervised machine learning and financial risk modeling to set to see whether the hypothesis of Borrowers with low annual income, higher loan-to-income percentages, shorter employment length, non-homeownership status, certain loan intents, prior default history, and shorter credit history are more likely to default on loans is a correct conclusion to draw from the dataset.

Important Metrics (i.e, variables)

person_age: measured in years. This variable reflects whether the individual securing a loan does have finacial stability as well as credit maturity (i.e., a history of paid off credit to buuild a good credit score) a good credit score indicates income stability. Yound Adults (i.e., 18-23) may have limited credit history increasing uncertanty in risk of loan defaulting.

person_income : represented in in annual income in the currency of US Dollar. An individual's income represents a good indicator for repayment capacity. However the individual's income should be looked from the perspective of loan size as outliers (i.e., high reported incomes) may lead to skewed distribution and errors in the data.

loan_amnt : is the loan ammount also represented in US Dollars. Larger loans tend to increase financial burden as repayment additionally often includes interest rates leading to repayment stress. A large loan amount would signifigantly increase the details explained above which would increase likelihood of loans being defaulted.

loan_int_rate : the annual interest rate of a loan represented in percentage,APR. higher interest rates than the avergae of 5-30 %increases the repayment cost and is often assigned to indivudals that are high-risk borrowers. These high interest rates would indicate that these individuals have poor credit history resulting in increased likelihood of loans being defaulted.

loan_percent_income : a ratio found by calculating loan amount / annual income. This metric allows to anaylse debt burden an indicate higher default

loan_grade : categorized from A-G , reflects lender assesed risk level aiding in summarizing multiple financial risk indicators into a single loan risk variable.

cb_person_default_on_file : binary variable (Y/N) indicate whether the indivdual who has taken out a loan before has had that loan defaulted. Previous history of defaulted loans is a strong predicator that furture loans would be defaulted as it reflects a repeated pattern of behavior rather than just financial instability.

There could be a concern of selction bias as the dataset has a likelihood of only having individuals who applied for loans which were then recorded into their system. It excludes indivduals who never applied for credit or who never made past the formal evaluation stage. This would lead to the conclusions that are drawn from this study may not be applicable to the general population.

There is a likelihood if institutional bias when it comes to variable such as loan_grade as well as loan_int_rate as the predictave models built on this dataset may largely reproduce clasic lending decisions rather than the individual factors leading to loan defaults.

The selection bias mentioned previous could result in class imbalance as well as in many modern credit bases non-defaulted loans are much greater than defaulted loans. This leads to the predictive models made may appear accurate to the conclusions with the hypothesis , but failing to detect high risk borrowers

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

# loading data
df = pd.read_csv('data/00-raw/credit_risk_dataset.csv')
print(f"Dataset Size: {df.shape[0]} rows and {df.shape[1]} columns")

# check and make sure strings are all capitalized so there are no discrepencies among categorical data
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.strip().str.title()
print("Data is tidy with categorical variables\n")

# Check to see if any data is missing
missing = df.isnull().sum()
print("Missing Data Count:\n", missing[missing > 0])

# Heatmap of missing data entries among variables
plt.figure(figsize=(10,4))
sns.heatmap(df.isnull(), cbar=False, yticklabels=False, cmap='viridis')
plt.title('Heatmap of missing entries (Yellow = Missing)')
plt.show()

# Remove extreme outliers and weird entries
suspicious_age = df[df['person_age'] > 100] 
suspicious_emp = df[df['person_emp_length'] > 60] 
logical_error = df[df['person_emp_length'] > (df['person_age'] - 14)]

print(f"\nFlagging Weird Entries:") 
print(f"   - Age > 100: {len(suspicious_age)} cases") 
print(f"   - Emp Length > 60: {len(suspicious_emp)} cases") 
print(f"   - Logical Errors (Emp > Age-14): {len(logical_error)} cases")

df = df[df['person_age'] < 100] 
df = df[df['person_emp_length'] < 60] 
df = df[df['person_emp_length'] <= (df['person_age'] - 14)]

# Clean missing data by using median imputation for interest rate since it is robust to right skewed 
# distributions. Missing employment length entries are filled with 0 assuming there is no history.
df['loan_int_rate'] = df['loan_int_rate'].fillna(df['loan_int_rate'].median()) 
df['person_emp_length'] = df['person_emp_length'].fillna(0)

# Cleaned up data numbers
print(f"\nFinal Cleaned Size: {df.shape[0]} rows") 
print(f"   Remaining missing values: {df.isnull().sum().sum()}")

# Summary statistics
print("\nSummary Statistics for Important Variables:") 
display(df[['person_age', 'person_income', 'loan_int_rate', 'person_emp_length']].describe())

# Exporting data file
os.makedirs('data/02-processed', exist_ok=True) 
df.to_csv('data/02-processed/credit_risk_clean.csv', index=False) 
print("\nClean data saved to data/02-processed/credit_risk_clean.csv!!!!")


## Ethics

[![Deon badge](https://img.shields.io/badge/ethics%20checklist-deon-brightgreen.svg?style=popout-square)](http://deon.drivendata.org/)

### A. Data Collection
 - [x] **A.1 Informed consent**: If there are human subjects, have they given informed consent, where subjects affirmatively opt-in and have a clear understanding of the data uses to which they consent?

> This project uses a publicly available, secondary dataset. Our team did not collect data directly from human subjects, so informed consent was not obtained by us. We assume informed consent and ethical data collection practices were handled by the original data providers.

 - [x] **A.2 Collection bias**: Have we considered sources of bias that could be introduced during data collection and survey design and taken steps to mitigate those?
> We acknowledge that collection bias may exist if certain borrower groups, such as higher income or more stable employment groups, are overrepresented in the dataset. This may limit how well the results generalize to all borrowers, so findings are interpreted cautiously.

 - [x] **A.3 Limit PII exposure**: Have we considered ways to minimize exposure of personally identifiable information (PII) for example through anonymization or not collecting information that isn't relevant for analysis?
> The dataset doesn't contain direct personally identifiable information such as names, addresses, or identification numbers. All variables used are anonymized and relevant to the analysis, minimizing exposure to PII.

 - [x] **A.4 Downstream bias mitigation**: Have we considered ways to enable testing downstream results for biased outcomes (e.g., collecting data on protected group status like race or gender)?

> The dataset doesn't include protected attributes such as race or gender, which limits our ability to test downstream bias across these groups. We acknowledge this limitation and avoid making claims about fairness across unobserved protected classes.

### B. Data Storage
- [x] **B.1 Data security**: Do we have a plan to protect and secure data (e.g., encryption at rest and in transit, access controls on internal users and third parties, access logs, and up-to-date software)?
> The data used in this project comes from open-source Kaggle datasets and does not contain sensitive or private information. We store the data locally and in GitHub/Colab environments, which already include standard access controls.

- [x] **B.2 Right to be forgotten**: Do we have a mechanism through which an individual can request their personal information be removed?
> Since the datasets are publicly available and fully anonymized, there is no direct way to trace data back to an individual. If a dataset were removed from Kaggle, we would stop using it.

- [x] **B.3 Data retention plan**: Is there a schedule or plan to delete the data after it is no longer needed?
> The data is only kept for the duration of this class project. After the project is complete, the local copies can be deleted since the original data remains publicly available on Kaggle.
### C. Analysis
- [x] **C.1 Missing perspectives**: Have we sought to address blindspots in the analysis through engagement with relevant stakeholders (e.g., checking assumptions and discussing implications with affected communities and subject matter experts)?
> The analysis is based on available loan and borrower data and does not directly include perspectives from borrowers or lenders. Because of this, some real-world factors behind loan default may not be fully captured, and results are interpreted with this limitation in mind.

- [x] **C.2 Dataset bias**: Have we examined the data for possible sources of bias and taken steps to mitigate or address these biases (e.g., stereotype perpetuation, confirmation bias, imbalanced classes, or omitted confounding variables)?
> Loan default is less common than repayment, which creates class imbalance in the dataset. Some potentially important factors, such as full credit history, are also not included, which may affect the results.

- [x] **C.3 Honest representation**: Are our visualizations, summary statistics, and reports designed to honestly represent the underlying data?
> Our visualizations and summary statistics are created to reflect the data accurately.

- [x] **C.4 Privacy in analysis**: Have we ensured that data with PII are not used or displayed unless necessary for the analysis?
> The dataset is anonymized and does not include personally identifiable information. All analysis is done at an aggregate level without exposing individual data points.

- [x] **C.5 Auditability**: Is the process of generating the analysis well documented and reproducible if we discover issues in the future?
> The analysis is documented in a notebook with clear code and explanations, making it possible to rerun or review the process if issues are found later.

### D. Modeling
- [x] **D.1 Proxy discrimination**: Have we ensured that the model does not rely on variables or proxies for variables that are unfairly discriminatory?
> The model uses variables such as income, employment length, home ownership, and loan intent. While these factors may indirectly relate to sensitive attributes, they are included to study patterns in loan default and not to make judgments about individuals.

- [x] **D.2 Fairness across groups**: Have we tested model results for fairness with respect to different affected groups (e.g., tested for disparate error rates)?
> Comparisons are made across available groups in the data, such as loan intent and home ownership status. However, fairness testing is limited because protected attributes like race or gender are not included.

- [x] **D.3 Metric selection**: Have we considered the effects of optimizing for our defined metrics and considered additional metrics?
> Since the outcome variable is binary, appropriate classification metrics are used. Class imbalance is also considered when evaluating model performance.

- [x] **D.4 Explainability**: Can we explain in understandable terms a decision the model made in cases where a justification is needed?
> Logistic regression is used so that the relationship between input variables and loan default can be explained in a clear and interpretable way.

- [x] **D.5 Communicate limitations**: Have we communicated the shortcomings, limitations, and biases of the model to relevant stakeholders in ways that can be generally understood?
> Limitations such as missing variables, potential bias, and dataset constraints are clearly discussed alongside the model results.

### E. Deployment
- [x] **E.1 Monitoring and evaluation**: Do we have a clear plan to monitor the model and its impacts after it is deployed (e.g., performance monitoring, regular audit of sample predictions, human review of high-stakes decisions, reviewing downstream impacts of errors or low-confidence decisions, testing for concept drift)?
> This model is not deployed in a real-world setting and is only used for this class project. If it were to be deployed, regular monitoring and review would be needed to ensure performance stays reasonable over time.

- [x] **E.2 Redress**: Have we discussed with our organization a plan for response if users are harmed by the results (e.g., how does the data science team evaluate these cases and update analysis and models to prevent future harm)?
> In a real application, loan decisions should involve human review and a way for individuals to question or appeal outcomes. This helps reduce harm from incorrect or unfair predictions.

- [x] **E.3 Roll back**: Is there a way to turn off or roll back the model in production if necessary?
> Any real deployment should allow the model to be turned off or replaced if issues such as bias or poor performance are discovered.

- [x] **E.4 Unintended use**: Have we taken steps to identify and prevent unintended uses and abuse of the model and do we have a plan to monitor these once the model is deployed?
> A possible unintended use would be applying the model directly to loan approval decisions. Since the model is built using limited features and open data, using it without proper safeguards could lead to unfair outcomes.

## Team Expectations 

* **Team Expectation 1: Communication Structure**  
We will communicate primarily through iMessage and expect responses within 24 hours. We will meet on Zoom weekly or as needed to discuss progress and next steps.

* **Team Expectation 2: Respectful Tone**  
We agree to communicate in a respectful, direct, and constructive manner. Feedback will focus on ideas and work rather than individuals.

* **Team Expectation 3: Decision Making**  
All major decisions will be made through group discussion and voting. If someone is unresponsive when a decision must be made, the group will move forward based on a majority vote and update them afterward.

* **Team Expectation 4: Task Assignment and Effort**  
Tasks will be assigned clearly and evenly. While members may focus on different areas, everyone is expected to contribute across all parts of the project, including coding, writing, and editing.

* **Team Expectation 5: Planning and Deadlines**  
We will set deadlines for each part of the project and review progress regularly during meetings. Plans may be adjusted as needed.

* **Team Expectation 6: Addressing Difficulties**  
If a team member is struggling with a task, they will communicate early so the group can adjust responsibilities and provide support.


## Project Timeline Proposal





| Meeting Date | Meeting Time | Completed Before Meeting | Discuss at Meeting |
|---|---|---|---|
| 2/2 | 1 PM Meeting | Brainstorm topic ideas; review project requirements | Discuss and decide on final project topic; assign group responsibilities; discuss hypothesis |
| 2/4 | Before 11:59 PM | Edit, finalize, and submit proposal | Proposal submitted |
| 2/11 to 2/16 | Meeting TBD | Finalize dataset and analysis approach | Discuss dataset details and ethics; prepare for Checkpoint 1 |
| 2/18 | Before 11:59 PM | NA | Review and submit Checkpoint 1 |
| 2/25 to 3/2 | Meeting TBD | Import and wrangle data; begin coding, exploratory data analysis, and visualizations | Review/Edit wrangling, code, EDA, and visualizations; discuss analysis approach |
| 3/4 | Before 11:59 PM | NA | Review and submit Checkpoint 2 |
| 3/11 to 3/16 | Meeting TBD | Continue analysis, coding, and visualizations; draft results and discussion | Discuss/edit analysis, code, visualizations, and prepare final project |
| 3/18 | Before 11:59 PM | NA | Turn in Final Project & Group Project Surveys |